In [10]:
# This code started as a direct copy from opencv
# https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

import numpy as np
import cv2 as cv

# Info about YOLO import and loading the Yolo Model
# https://docs.ultralytics.com/tasks/classify/#train

# Added this to import YOLO
from ultralytics import YOLO

# Load the YOLOv8 model, this is loading our custom trained weights for our model.
model = YOLO("best_v2.pt")
 
cap = cv.VideoCapture(0)

# Stored captured text
captured_text = []

# Stores captured confidence
captured_confidence = []

# Set a threshold for the sign to register
confidence_requirement = 0.80

# Counts the number of consecutive significant signs
count = 0

# Counts the number of consecutive insignificant signs
noise_count = 0

# Keeps track of the last sign
last = None

if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # These two lines are found here https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html
    # They reduce the size of the video
    
    ret = cap.set(cv.CAP_PROP_FRAME_WIDTH,240)
    ret = cap.set(cv.CAP_PROP_FRAME_HEIGHT,240)
    
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    cv.imshow('frame', gray)
    
    # If you scroll to the very bottom of this link
    # https://docs.ultralytics.com/modes/predict/#thread-safe-inference
    # You will find the next 3 lines of code which I took from their and applied to this similar example
    
    # Run YOLOv8 inference on the frame
    results = model(frame)
    top_class = results[0].probs.top1
    top_confidence = results[0].probs.top1conf  # Get confidence of the top-class prediction
    
    # If the confidence of the sign is above the threshold 
    if (top_confidence >= confidence_requirement):
        
        # If the top_class is the last class
        # hence it is consecutive increase count by 1.
        if (top_class==last):
            
            count = count+1
        
        # If the top_class is not the last class
        # it is  a new class restart counter 
        else:
    
            count=1
            
        # If there are 3 consecutive significant signs track it
        if (count == 3):
        
            captured_text.append(top_class)
            captured_confidence.append(top_confidence)
        
        # Set last to be the top_class
        last = top_class
        
        # Set the noise counter to 0 since this is not noise
        noise_count = 0 
    
    # If the confidence of the current sign is not enough for the threshold increase noise counter
    else:
        
        noise_count=noise_count+1
    
    # If there are three consecutive insignificant signs
    # Reset count, allowing another consecutive sign to be registerred for instance (A,A)
    # Reset the noise counter
    if noise_count == 3:
        count = 0
        noise_count = 0
    
    # Visualize the results on the frame
    annotated_frame = results[0].plot()
    
    # Display the annotated frame
    cv.imshow("YOLOv8 Inference", annotated_frame)

    if cv.waitKey(1) == ord('q'):
        print("Captured top class: ", captured_text)
        
        # ASCII ref
        #https://simple.m.wikipedia.org/wiki/File:ASCII-Table-wide.svg
        
        # https://stackoverflow.com/questions/3673428/convert-int-to-ascii-and-back-in-python
        # Integer to ASCII
        
        translated_text = []
        for i in captured_text:
            
            if i<26:
                translated_text.append(chr(i+65))
            elif i==26:
                translated_text.append("del")
            elif i==27:
                translated_text.append("nothing")
            elif i==28:
                translated_text.append("space")
        
        for i in range(len(translated_text)):
            
            translated_text[i] = (translated_text[i], captured_confidence[i].item())
        
        print("Translated text: ", translated_text)
        break
        
        
 
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()


0: 224x224 nothing 0.04, X 0.04, Y 0.04, W 0.04, P 0.04, 12.7ms
Speed: 10.5ms preprocess, 12.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 M 0.23, I 0.17, G 0.11, N 0.10, P 0.07, 12.8ms
Speed: 6.5ms preprocess, 12.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 M 0.18, G 0.15, I 0.13, N 0.10, A 0.07, 10.7ms
Speed: 5.9ms preprocess, 10.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 G 0.22, I 0.11, S 0.11, P 0.09, M 0.08, 11.4ms
Speed: 4.9ms preprocess, 11.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 G 0.20, S 0.08, I 0.08, M 0.07, E 0.06, 9.8ms
Speed: 6.4ms preprocess, 9.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 G 0.20, S 0.09, M 0.08, E 0.07, I 0.06, 11.7ms
Speed: 6.6ms preprocess, 11.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 M 0.20, G 0.16, P 0.12, I 0.12, N 0.11, 8.6ms
S